Some of the features that has and hasn't been done:

- [X] Display points from all views simultaneously.
- [X] Give color according to vehicle type
- [X] Display the CCTV views
- [X] Display the bounding boxes to CCTV views.

In [2]:
import cv2
import numpy as np
import pandas as pd
import os
import time

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.pyplot as plt

region_data = pd.read_csv('regions.csv')
region_data

,cctv_folder_parent,map_image_name
0,SIMPANG 3 DPR,SIMPANG 3 DPR.jpeg
1,SIMPANG 4 CUNGKING,SIMPANG 4 CUNGKING.jpeg
2,SIMPANG 4 PATUNG KUDA,SIMPANG 4 PATUNG KUDA.jpeg
3,SIMPANG 5 PUSAT KOTA,SIMPANG 5 BANYUWANGI.jpeg


In [3]:
cctv_data = pd.read_csv('cctv_views.csv')
cctv_data

,region_name,cctv_view_name
0,SIMPANG 3 DPR,dpr selatan
1,SIMPANG 3 DPR,dpr timur
2,SIMPANG 3 DPR,dpr utara
3,SIMPANG 4 PATUNG KUDA,patungkuda barat
4,SIMPANG 4 PATUNG KUDA,patungkuda selatan
5,SIMPANG 4 PATUNG KUDA,patungkuda utara
6,SIMPANG 4 CUNGKING,cungking selatan
7,SIMPANG 4 CUNGKING,cungking timur
8,SIMPANG 4 CUNGKING,cungking utara


In [4]:
# Define a CCTV path of image whose region has been detected
# Define the map path as well.

# You can freely change this depending on the condition of your database.
region_name = "SIMPANG 4 PATUNG KUDA"
cctv_range = {'start':400, 'end': 3300}
cctv_time = '1200'

region_cctv_views = cctv_data[cctv_data['region_name'] == region_name]['cctv_view_name'].values.tolist()

region_map_path = region_data[region_data['cctv_folder_parent'] == region_name]
region_map_path = 'map/' + region_map_path['map_image_name'].values.tolist()[0]

region_coords = pd.read_csv('coords/cctv/{:}.csv'.format(region_name))

print("CCTV VIEWS :", region_cctv_views)
print("MAP PATH   :", region_map_path)
print("REGION COORDINATES:")
region_coords

CCTV VIEWS : ['patungkuda barat', 'patungkuda selatan', 'patungkuda utara']
MAP PATH   : map/SIMPANG 4 PATUNG KUDA.jpeg
REGION COORDINATES:


,view,pt1a,pt1b,pt1c,pt1d,pt2a,pt2b,pt2c,pt2d,pt3a,pt3b,pt3c,pt3d
0,patungkuda selatan,97 273,363 236,1074 476,187 770,0 0,1080 0,1080 720,0 720,400 717,295 644,392 513,492 587
1,patungkuda barat,132 283,396 262,1080 505,211 720,0 0,1080 0,1080 720,0 720,364 296,475 236,578 379,458 450
2,patungkuda utara,510 170,668 168,664 437,0 390,0 0,1080 0,1080 720,0 720,648 167,748 239,598 437,497 362


In [5]:
map_image = cv2.imread(region_map_path)

In [6]:
def get_points(pandas_frame, view_name):
    polygons = {}
    row = pandas_frame[pandas_frame['view'] == view_name]
    point_id = ['a', 'b', 'c', 'd']
    polygon_id = ['1', '2', '3']
    for i in polygon_id:
        polygon = []
        for j in point_id:
            point_name = 'pt'+ i + j
            point_array = [int(p) for p in row[point_name].values.tolist()[0].split()]
            polygon.append(point_array)
        polygons[i] = np.float32(polygon)
    return polygons

In [7]:
def warpPerspectivePoints(pts, M):
    n = pts.shape[0]
    trans = np.hstack((pts, np.ones((n,1)))) # Add ones at the bottom of matrix
    trans = np.matmul(M, np.transpose(trans))
    return np.transpose(trans[0:2,:] / trans[2:])

##### Pseudocode:

For every CCTV views:

1. Eliminate non-vehicles point (can be obtained from the "object" header)
   For example, vehicles: truck, car, motorbike
1. For every bounding box vehicles:
   1. Find the bounding box center point
   1. Map the point according to transformation matrix M
1. Now, for displaying purposes:
   1. Draw the map and all the transformed points
   1. In new window, draw the CCTV image with boxes

In [8]:
coco_vehicle_labels = ['bicycle', 'car', 'motorbike', 'bus', 'truck']

def eliminate_non_vehicle(series):
    detections = series['object_probs'].split(' ')
    if detections[0] in coco_vehicle_labels:
        return detections[0]
    return None

ord_q = ord('q') # For quit button
ord_p = ord('p') # For pause button

def pause_and_is_finished():
    # Pauses and quit button purposes
    k = cv2.waitKey(25) & 0xFF
    finished = k == ord_q
    if k == ord_p:
        while True:
            k = cv2.waitKey(25) & 0xFF
            finished = k == ord_q
            if k == ord_p or k == ord_q:
                break
    return finished

def generate_equal_rgb_color_distance(n):
    colors = []
    for i in range(n):
        color = np.uint8(cv2.cvtColor(np.float32([[[i*360/n, 0.8, 0.9]]]), cv2.COLOR_HSV2RGB)[0, 0] * 255)
        colors.append(color)
    return colors

coco_colors = generate_equal_rgb_color_distance(len(coco_vehicle_labels))
coco_colors

def get_color_from_class(row):
    class_name = row.split()[0]
    class_index = coco_vehicle_labels.index(class_name)
    return coco_colors[class_index]

In [9]:
# Let's define a dictionary to store variables.
# These variables include precomputed value as well, such as
# the transformation matrix.

objs = {}
for i in range(len(region_cctv_views)):
    obj_dict = {}
    
    rctv = region_cctv_views[i]
    path_cctv_view = './frame/cctv_{:}_{:} {:}_x264.mp4/'.format(region_name, rctv, cctv_time)
    
    polygons = get_points(region_coords, rctv)

    pts1 = polygons['1']
    pts2 = polygons['3']

    # Get transformation matrix
    M = cv2.getPerspectiveTransform(pts1,pts2)
    pts_transform = np.int32(warpPerspectivePoints(pts1, M))    

    # Get detections CSV file name length
    # It is assumed that all files has the same name length.
    file_ext_first_file = os.path.splitext(os.listdir(path_cctv_view)[0])
    filename_length = len(file_ext_first_file[0])
    
    # Populate variables to the object dictionary.    
    obj_dict['rctv'] = rctv
    obj_dict['path_cctv_view'] = path_cctv_view
    obj_dict['pts1'] = pts1
    obj_dict['pts2'] = pts2
    obj_dict['filename_length'] = filename_length
    obj_dict['M'] = M
    objs[i] = obj_dict

In [10]:
# Resize the map image so that it is faster to display
RESIZE_CONSTANT = 0.55
CCTV_VIEW_RESIZE = 0.25

# Other constants
TEXT_POSITION    = np.int32(np.array([25, 40]) * RESIZE_CONSTANT)
TEXT_SIZE        = 1
TEXT_COLOR       = (0, 0, 0)
TEXT_BOLDNESS    = round(2 * RESIZE_CONSTANT)
POINT_SIZE       = round(10 * RESIZE_CONSTANT)
BOX_OUTLINE_SIZE = round(10 * CCTV_VIEW_RESIZE)
    
# Display the legends
MAX_CHARACTER_LENGTH   = max([len(c) for c in coco_vehicle_labels] + [len('Legends:')]) + 3
CHARACTER_WIDTH        = 15
CHARACTER_HEIGHT       = 35
LEGEND_CANVAS_SIZE_X   = MAX_CHARACTER_LENGTH * CHARACTER_WIDTH + 40 # Add 40 px more for color label
LEGEND_CANVAS_SIZE_Y   = len(coco_vehicle_labels) + 2 # Add 2 more rows for "Legends" label and "Total:" label
LEGEND_CANVAS_SIZE_Y   = LEGEND_CANVAS_SIZE_Y * CHARACTER_HEIGHT + 20
CIRCLE_SIZE            = 10
LEGEND_FONT_SIZE       = 0.8
LEGEND_LABEL_FONT_SIZE = 0.6
LEGEND_FONT_COLOR      = (0,0,0)
LEGEND_LABEL_OUTLINE   = 0
LEGEND_TITLE_POS       = (15, 30)
LEGEND_TITLE_OUTLINE   = 2
LEGEND_TOTAL_TEXT_POS  = (15, (len(coco_vehicle_labels) + 1) * CHARACTER_HEIGHT + 30)

def display_legends(vehicle_count):
    total_vehicle = np.sum(vehicle_count)
    legend_canvas = np.uint8(np.ones((LEGEND_CANVAS_SIZE_Y, LEGEND_CANVAS_SIZE_X, 3)) * 255)
    cv2.putText(legend_canvas, 'LEGENDS:', LEGEND_TITLE_POS, cv2.FONT_HERSHEY_SIMPLEX,
                LEGEND_FONT_SIZE, LEGEND_FONT_COLOR, LEGEND_TITLE_OUTLINE, cv2.LINE_AA)

    for i, label in enumerate(coco_vehicle_labels):
        LABEL_CIRCLE_POS = (25, 35 * (i+1) + 25)
        LABEL_TEXT_POS   = (45, 35 * (i+1) + 30)
        LABEL_TEXT       = '{:} ({:})'.format(label, vehicle_count[i])
        
        cv2.putText(legend_canvas, LABEL_TEXT, LABEL_TEXT_POS,
                    cv2.FONT_HERSHEY_SIMPLEX, LEGEND_LABEL_FONT_SIZE,
                    LEGEND_FONT_COLOR, LEGEND_LABEL_OUTLINE, cv2.LINE_AA)
        cv2.circle(legend_canvas, LABEL_CIRCLE_POS, CIRCLE_SIZE, coco_colors[i].tolist(), -1)
    cv2.putText(legend_canvas, 'Total: {:}'.format(total_vehicle), LEGEND_TOTAL_TEXT_POS,
                cv2.FONT_HERSHEY_SIMPLEX, LEGEND_FONT_SIZE, LEGEND_FONT_COLOR,
                LEGEND_LABEL_OUTLINE, cv2.LINE_AA)

    cv2.imshow('Legend', legend_canvas)
    
def display_vehicle_total_count(vehicle_total_count):
    fig = Figure()
    canvas = FigureCanvas(fig)
    ax = fig.gca()
    ax.plot(vehicle_total_count)
    ax.set_title('Vehicle count')

    canvas.draw()
    total_vehicle_image = np.fromstring(canvas.tostring_rgb(), dtype='uint8')
    width, height = np.int32(fig.get_size_inches() * fig.get_dpi())
    total_vehicle_image = np.reshape(total_vehicle_image, (height, width, 3))
    total_vehicle_image = cv2.cvtColor(total_vehicle_image, cv2.COLOR_RGB2BGR)
    
    cv2.imshow('Vehicle count', total_vehicle_image)

In [ ]:
map_image_resized = cv2.resize(map_image, (0,0), fx=RESIZE_CONSTANT, fy=RESIZE_CONSTANT)
force_quit = False
total_vehicle_over_time = np.int32([])

try:
    for i in range(cctv_range['start'], cctv_range['end'] + 1):
        
        # Copy the image. Later we will draw something.
        # We don't want the original to be overwritten.
        new_map_image = map_image_resized.copy()
        image_cctvs = np.array([[[]]])
        
        # Give frame number
        cv2.putText(new_map_image, 'Frame number: {:}'.format(i),
                    tuple(TEXT_POSITION.tolist()),
                    cv2.FONT_HERSHEY_SIMPLEX, TEXT_SIZE * RESIZE_CONSTANT,
                    TEXT_COLOR, TEXT_BOLDNESS, cv2.LINE_AA)
        
        # Initiate vehicle counter
        vehicle_count = np.int32(np.zeros(len(coco_vehicle_labels)))
        for j in range(len(region_cctv_views)):
            obj = objs[j]
            
            # Print current progress (for debug purposes)
            # print('\r' + str(i) + '-' + obj['rctv'] + '   ', end='')
            
            M = obj['M']
            filename_length = obj['filename_length']
            path_cctv_view = obj['path_cctv_view']
            
            csv_file_name = str(i).zfill(filename_length)
            csv_path = '{:}/{:}.csv'.format(path_cctv_view, csv_file_name)
            
            try: 
            # Read CSV
                objects = pd.read_csv(csv_path)

                n_detection = objects.shape[0]
                if (n_detection > 0):
                    # Eliminate non-vehicles object
                    vehicle_labels = objects.apply(lambda r: eliminate_non_vehicle(r), axis=1)
                    objects = objects[~vehicle_labels.isna()]
                    vehicle_labels = vehicle_labels[~vehicle_labels.isna()]

                    # Count the vehicles
                    vehicle_count += [(vehicle_labels == l).sum() for l in coco_vehicle_labels]

                n_detection = objects.shape[0]
                if (n_detection > 0):
                    # For every bounding box vehicles, find the bounding box center of mass
                    centers = objects.apply(lambda r: [
                                      (r['left'] + r['right'])/2,
                                      (r['top'] + r['bottom'])/2
                                      ], axis=1)

                    # Transform the points
                    points_np = np.array([c for c in centers])
                    points_transform = np.float64(warpPerspectivePoints(points_np, M))

                    # Resize the points and the map image so that it is faster to display
                    points_transform *= RESIZE_CONSTANT
                    points_transform = np.int32(points_transform)

                    # Get colors
                    colors = objects['object_probs'].apply(lambda row: get_color_from_class(row)).values.tolist()

                    # Map the points to the image
                    for k, pts_t in enumerate(points_transform):
                        cv2.circle(new_map_image, tuple(pts_t), POINT_SIZE, colors[k].tolist(), -1)
            except Exception:
                print(str(i) + " csv not found")
            
            # Read CCTV image, replacing CSV path with JPG path
            cctv_view_image = cv2.imread(csv_path[:-3] + 'jpg')
            cctv_view_image = cv2.resize(cctv_view_image, (0,0), fx=CCTV_VIEW_RESIZE, fy=CCTV_VIEW_RESIZE)
            
            # Draw bounding boxes
            if (n_detection > 0):
                boxes = objects.apply(lambda r: [
                                  [r['left'], r['top']],
                                  [r['right'], r['bottom']]
                                  ], axis=1)
                # Resize the boxes
                boxes = np.int32(np.int32([k for k in boxes.values]) * CCTV_VIEW_RESIZE)
                for k, box in enumerate(boxes):
                    cv2.rectangle(cctv_view_image, tuple(box[0]), tuple(box[1]), tuple(colors[k].tolist()),
                                  BOX_OUTLINE_SIZE)
            
            if (j == 0):
                image_cctvs = cctv_view_image
            else:
                image_cctvs = np.concatenate((image_cctvs, cctv_view_image), axis=0)
        
        total_vehicle_over_time = np.append(total_vehicle_over_time, [np.sum(vehicle_count)])
        
        cv2.imshow('Image', new_map_image)
        cv2.imshow('Image CCTV', image_cctvs)
        display_legends(vehicle_count)
        display_vehicle_total_count(total_vehicle_over_time)
        
        if pause_and_is_finished():
            force_quit = True
            break
    
    if not force_quit:
        cv2.putText(new_map_image, 'Frame number: {:} ~FINISHED!'.format('      '),
                    tuple(TEXT_POSITION.tolist()),
                    cv2.FONT_HERSHEY_SIMPLEX, TEXT_SIZE * RESIZE_CONSTANT,
                    TEXT_COLOR, TEXT_BOLDNESS, cv2.LINE_AA)
        cv2.imshow('Image', new_map_image)
        
        while not pause_and_is_finished():
            pass
finally:
    cv2.destroyAllWindows()

/home/agas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


404 csv not found
404 csv not found
404 csv not found
421 csv not found
421 csv not found
421 csv not found
427 csv not found
427 csv not found
427 csv not found
446 csv not found
446 csv not found
446 csv not found
450 csv not found
450 csv not found
450 csv not found
451 csv not found
451 csv not found
451 csv not found
455 csv not found
455 csv not found
455 csv not found
464 csv not found
464 csv not found
464 csv not found
468 csv not found
468 csv not found
468 csv not found
469 csv not found
469 csv not found
469 csv not found
473 csv not found
473 csv not found
473 csv not found
478 csv not found
478 csv not found
478 csv not found
481 csv not found
481 csv not found
481 csv not found
484 csv not found
484 csv not found
484 csv not found
491 csv not found
491 csv not found
491 csv not found
497 csv not found
497 csv not found
497 csv not found
502 csv not found
502 csv not found
502 csv not found
503 csv not found
503 csv not found
503 csv not found
505 csv not found
505 csv no

1377 csv not found
1377 csv not found
1379 csv not found
1379 csv not found
1379 csv not found
1380 csv not found
1380 csv not found
1380 csv not found
1381 csv not found
1381 csv not found
1381 csv not found
1385 csv not found
1385 csv not found
1385 csv not found
1392 csv not found
1392 csv not found
1392 csv not found
1397 csv not found
1397 csv not found
1397 csv not found
1415 csv not found
1415 csv not found
1415 csv not found
1416 csv not found
1416 csv not found
1416 csv not found
1420 csv not found
1420 csv not found
1420 csv not found
1425 csv not found
1425 csv not found
1425 csv not found
1427 csv not found
1427 csv not found
1427 csv not found
1433 csv not found
1433 csv not found
1433 csv not found
1435 csv not found
1435 csv not found
1435 csv not found
1438 csv not found
1438 csv not found
1438 csv not found
1440 csv not found
1440 csv not found
1440 csv not found
1447 csv not found
1447 csv not found
1447 csv not found
1459 csv not found
1459 csv not found
1459 csv not

2282 csv not found
2282 csv not found
2282 csv not found
2287 csv not found
2287 csv not found
2287 csv not found
2290 csv not found
2290 csv not found
2290 csv not found
2293 csv not found
2293 csv not found
2293 csv not found
2294 csv not found
2294 csv not found
2294 csv not found
2301 csv not found
2301 csv not found
2301 csv not found
2308 csv not found
2308 csv not found
2308 csv not found
2309 csv not found
2309 csv not found
2309 csv not found
2315 csv not found
2315 csv not found
2315 csv not found
2317 csv not found
2317 csv not found
2317 csv not found
2320 csv not found
2320 csv not found
2320 csv not found
2322 csv not found
2322 csv not found
2322 csv not found
2325 csv not found
2325 csv not found
2325 csv not found
2331 csv not found
2331 csv not found
2331 csv not found
2332 csv not found
2332 csv not found
2332 csv not found
2338 csv not found
2338 csv not found
2338 csv not found
2339 csv not found
2339 csv not found
2339 csv not found
2352 csv not found
2352 csv not

3228 csv not found
3228 csv not found
3228 csv not found
3229 csv not found
3229 csv not found
3229 csv not found
3230 csv not found
3230 csv not found
3230 csv not found
3235 csv not found
3235 csv not found
3235 csv not found
3237 csv not found
3237 csv not found
3237 csv not found
3242 csv not found
3242 csv not found
3242 csv not found
3264 csv not found
3264 csv not found
3264 csv not found
3265 csv not found
3265 csv not found
3265 csv not found
3269 csv not found
3269 csv not found
3269 csv not found
3271 csv not found
3271 csv not found
3271 csv not found
3296 csv not found
3296 csv not found
3296 csv not found
3299 csv not found
3299 csv not found
3299 csv not found


---
##### From this point, the cells below are used for sandbox purposes only.

In [ ]:
# Try for single video frames first
try:
    rctv = region_cctv_views[2]
    path_cctv_view = './frame/cctv_{:}_{:} {:}_x264.mp4/'.format(region_name, rctv, cctv_time)
    polygons = get_points(region_coords, rctv)

    pts1 = polygons['1']
    pts2 = polygons['3']

    print("VIEW:", rctv)
    print("PTS 1:", str(pts1).replace("\n", ""))
    print("PTS 2:", str(pts2).replace("\n", ""))

    # Get transformation matrix
    M = cv2.getPerspectiveTransform(pts1,pts2)
    pts_transform = np.int32(warpPerspectivePoints(pts1, M))
    
    # Draw polygon containing the transformed region
    overlay_transform = np.uint8(np.ones(map_image.shape)) * 255
    cv2.fillPoly(overlay_transform, [np.int32(pts_transform)], (0, 0, 255))
    cv2.fillPoly(map_image_filled, [np.int32(pts_transform)], (255, 0, 0))
    
    print(pts_transform.tolist())

    # Get file name length (it is assumed to be same for all the video frames)
    file_ext_first_file = os.path.splitext(os.listdir(path_cctv_view)[0])
    print("FIRST FILE:", file_ext_first_file)
    filename_length = len(file_ext_first_file[0])

    coco_vehicle_labels = ['bicycle', 'car', 'motorbike', 'bus', 'truck']

    def strip(text):
        try:
            return text.strip()
        except AttributeError:
            return text

    # Try for single frame video first
    # Read CSV detection sequentially
    
    for i in range(45, cctv_range['end'] + 1):
        new_map_image = map_image_filled.copy()
        csv_file_name = str(i).zfill(filename_length)
        csv_path = '{:}/{:}.csv'.format(path_cctv_view, csv_file_name)

        # Read CSV
        objects = pd.read_csv(csv_path)
        print()
        print("OBJECTS:")
        print(objects)
        print()

        # Eliminate non-vehicles object
        def eliminate_non_vehicle(series):
            detections = series['object_probs'].split(' ')
            for dtc in detections:
                if dtc in coco_vehicle_labels:
                    return True
            return False
        
        objects = objects[objects.apply(lambda r: eliminate_non_vehicle(r), axis=1)]

        # Change string columns into numeric
        # -- Yeah, a little mistake I did when outputting CSV in darknet :( 
        #cols = ['left', 'right', 'top', 'bottom']    
        #for j in cols:
        #    objects[j] = pd.to_numeric(objects[j])

        # For every bounding box vehicles, find the bounding box center point    
        centers = objects.apply(lambda r: [
                          (r['left'] + r['right'])/2,
                          (r['top'] + r['bottom'])/2
                          ], axis=1)
        print("CENTERS:", str(centers).replace("\n", "").replace("  ", " "))

        # Transformation the points
        points_np = np.array([i for i in centers])
        points_transform = np.int32(warpPerspectivePoints(points_np, M))
        print("CENTER TRANSFORMED", str(points_transform).replace("\n", ""))

        # Map the points to the image
        for i in points_transform:
            cv2.circle(new_map_image, tuple(i), 9, (0,255,0), -1)
        cv2.imshow('Image2', overlay_transform)

        cv2.imshow('Image', new_map_image)
        while True:
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        break
finally:
    cv2.destroyAllWindows()

In [ ]:
# Now let's try it for all video frames.
try:
    rctv = region_cctv_views[1]
    path_cctv_view = './frame/cctv_{:}_{:} {:}_x264.mp4/'.format(region_name, rctv, cctv_time)
    polygons = get_points(region_coords, rctv)

    pts1 = polygons['1']
    pts2 = polygons['3']

    # Get transformation matrix
    M = cv2.getPerspectiveTransform(pts1,pts2)
    pts_transform = np.int32(warpPerspectivePoints(pts1, M))
    
    # Draw polygon containing the transformed region
    overlay_transform = np.uint8(np.ones(map_image.shape)) * 255
    map_image_filled = map_image.copy()
    cv2.fillPoly(overlay_transform, [np.int32(pts_transform)], (0, 0, 255))

    # Get file name length (it is assumed to be same for all the video frames)
    file_ext_first_file = os.path.splitext(os.listdir(path_cctv_view)[0])
    filename_length = len(file_ext_first_file[0])

    coco_vehicle_labels = ['bicycle', 'car', 'motorbike', 'bus', 'truck']

    def strip(text):
        try:
            return text.strip()
        except AttributeError:
            return text

    # Read CSV detection sequentially    
    for i in range(cctv_range['start'], cctv_range['end'] + 1):
        print('\r' + str(i), end='')
        new_map_image = map_image_filled.copy()
        csv_file_name = str(i).zfill(filename_length)
        csv_path = '{:}/{:}.csv'.format(path_cctv_view, csv_file_name)
        
        # Read CSV
        objects = pd.read_csv(csv_path)
        
        # Do not progress if nothing is detected
        n_detection = objects.shape[0]
        if (n_detection > 0):
            
            # Eliminate non-vehicles object
            def eliminate_non_vehicle(series):
                detections = series['object_probs'].split(' ')
                for dtc in detections:
                    if dtc in coco_vehicle_labels:
                        return True
                return False

            objects = objects[objects.apply(lambda r: eliminate_non_vehicle(r), axis=1)]
            
        n_detection = objects.shape[0]
        if (n_detection > 0):
            # For every bounding box vehicles, find the bounding box center point    
            centers = objects.apply(lambda r: [
                              (r['left'] + r['right'])/2,
                              (r['top'] + r['bottom'])/2
                              ], axis=1)

            # Transformation the points
            points_np = np.array([i for i in centers])
            points_transform = np.int32(warpPerspectivePoints(points_np, M))
            
            # Map the points to the image
            for i in points_transform:
                cv2.circle(new_map_image, tuple(i), 9, (0,255,0), -1)

        cv2.imshow('Image', new_map_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
finally:
    cv2.destroyAllWindows()

In [ ]:
rgbimg = cv2.cvtColor(np.float64([0, 0, 100]), cv2.COLOR_HSV2RGB)

In [ ]:
cv2.cvtColor(new_map_image, cv2.COLOR_BGR2HSV)

In [ ]:
for i in range(0, 360):
    print('\r', i, np.uint8(cv2.cvtColor(np.float32([[[i, 1, 1]]]), cv2.COLOR_HSV2RGB)[0, 0] * 255), end='           ')
    time.sleep(0.05)

In [ ]:
type(np.uint8([[new_map_image[0,0]]]).tolist()[0][0])

In [ ]:
np.uint8([[[360,255,0 ]]])